# Multi-tenant Chat with Papers - Setup
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_RESOURCE_NAME = os.getenv("AZURE_RESOURCE_NAME")

print(WEAVIATE_HTTP_URL)
print(WEAVIATE_GRPC_URL)
print(AZURE_BASE_URL)
print(AZURE_RESOURCE_NAME)

## Connect to Weaviate

In [ ]:
import weaviate
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,
)

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="main_vector",

            deployment_id="text-embedding-3-small",
            resource_name=AZURE_RESOURCE_NAME,
            base_url=AZURE_BASE_URL,
        )
    ],

    # supported models: https://weaviate.io/developers/weaviate/model-providers/openai/generative#available-models
    generative_config=Configure.Generative.azure_openai(
        deployment_id="gpt-4o-mini-0718",
        resource_name=AZURE_RESOURCE_NAME, # select your generative model
        base_url=AZURE_BASE_URL 
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("Papers")
papers.tenants.get()

## Close the client

In [ ]:
client.close()